# Example 2 - Custom Approach

This notebook demonstrates how to use the TX2 dashboard with a sequence classification transformer using the custom approach as described in the "Basic Usage" docs. To demonstrate, we simply take the default functions for each and manually define them.

In [1]:
%cd -q ..
%load_ext autoreload
%autoreload 2
%matplotlib inline

We enable logging to view the output from `wrapper.prepare()` further down in the notebook. (It's a long running function, and logs which step it's on.)

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
import numpy as np
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, BertTokenizer
import pandas as pd

In this example notebook, we use the 20 newsgroups dataset, which can be downloaded through sklearn via below:

In [4]:
from sklearn.datasets import fetch_20newsgroups

train_data = fetch_20newsgroups(subset='train')
test_data = fetch_20newsgroups(subset='test')

Defined below is a simple sequence classification model with a variable for the language model itself `l1`. Since it is a BERT model, we take the sequence embedding from the `[CLS]` token (via `output_1[0][:, 0, :])`) and pipe that into the linear layer.

In [5]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained("bert-base-cased")
        self.l2 = torch.nn.Linear(768, 20)

    def forward(self, ids, mask):
        output_1= self.l1(ids, mask)
        output = self.l2(output_1[0][:, 0, :]) # use just the [CLS] output embedding
        return output
    
model = BERTClass()
model.to("cuda")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Some simplistic data cleaning, and putting all data into dataframes for the wrapper

In [6]:
def clean_text(text):
    text = text[text.index("\n\n")+2:]
    text = text.replace("\n", " ")
    text = text.replace("    ", " ")
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.strip()
    return text

In [7]:
train_rows = []
for i in range(len(train_data["data"])):
    row = {}
    row["text"] = clean_text(train_data["data"][i])
    row["target"] = train_data['target'][i]
    if row["text"] == "" or row["text"] == " ": continue
    train_rows.append(row)
train_df = pd.DataFrame(train_rows)

In [8]:
test_rows = []
for i in range(len(test_data["data"])):
    row = {}
    row["text"] = clean_text(test_data["data"][i])
    row["target"] = test_data['target'][i]
    if row["text"] == "" or row["text"] == " ": continue
    test_rows.append(row)
test_df = pd.DataFrame(test_rows)

## Training

This section minimally trains the classification and language model - nothing fancy here, just to give the dashboard demo something to work with. Most of this is similar to the huggingface tutorial notebooks.

In [9]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-05)

In [10]:
class EncodedSet(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        print(self.len)

    def __getitem__(self, index):
        text = str(self.data.text[index])
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.target[index], dtype=torch.long)
        }

    def __len__(self):
        return self.len
    
train_set = EncodedSet(train_df, tokenizer, 256)
test_set = EncodedSet(test_df[:1000], tokenizer, 256)

train_params = {'batch_size': 16,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 2,
                'shuffle': True,
                'num_workers': 0
                }

# put everything into data loaders
train_loader = DataLoader(train_set, **train_params)
test_loader = DataLoader(test_set, **test_params)

11296
1000


In [11]:
device = "cuda"
def train(epoch):
    model.train()

    
    loss_history = []
    for _,data in tqdm(enumerate(train_loader, 0), total=len(train_loader), desc=f"Epoch {epoch}"):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask).squeeze()

        optimizer.zero_grad()
        loss = loss_function(outputs, targets)
        if _%100==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        loss_history.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         torch.cuda.empty_cache()
    return loss_history

In [12]:
losses = []
for epoch in range(1):
    losses.extend(train(epoch))

Epoch 0:   0%|          | 0/706 [00:00<?, ?it/s]

/home/81n/miniconda3/envs/adapd_nlp_hf/lib/python3.8/site-packages/transformers-4.1.1-py3.8.egg/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).


Epoch: 0, Loss:  3.045555830001831
Epoch: 0, Loss:  2.6691184043884277
Epoch: 0, Loss:  1.8494617938995361
Epoch: 0, Loss:  0.3816988170146942
Epoch: 0, Loss:  1.6430333852767944
Epoch: 0, Loss:  0.8961666822433472
Epoch: 0, Loss:  1.0292330980300903
Epoch: 0, Loss:  0.6516829133033752


The wrapper uses an `encodings` dictionary for various labels/visualizations, and can be set up with something similar to:

In [13]:
encodings = {}
for index, entry in enumerate(train_data["target_names"]):
    encodings[entry] = index
encodings

{'alt.atheism': 0,
 'comp.graphics': 1,
 'comp.os.ms-windows.misc': 2,
 'comp.sys.ibm.pc.hardware': 3,
 'comp.sys.mac.hardware': 4,
 'comp.windows.x': 5,
 'misc.forsale': 6,
 'rec.autos': 7,
 'rec.motorcycles': 8,
 'rec.sport.baseball': 9,
 'rec.sport.hockey': 10,
 'sci.crypt': 11,
 'sci.electronics': 12,
 'sci.med': 13,
 'sci.space': 14,
 'soc.religion.christian': 15,
 'talk.politics.guns': 16,
 'talk.politics.mideast': 17,
 'talk.politics.misc': 18,
 'talk.religion.misc': 19}

## TX2

This section shows how to put everything into the TX2 wrapper to get the dashboard widget displayed.

In [14]:
from tx2.wrapper import Wrapper
from tx2.dashboard import Dashboard

In [15]:
from tx2 import utils

def custom_encoding_function(text):
    encoded = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=256,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=True,
    )
    return {
        "input_ids": torch.tensor(encoded["input_ids"], device=utils.get_device()),
        "attention_mask": torch.tensor(encoded["attention_mask"], device=utils.get_device()),
    }

def custom_classification_function(inputs):
    return torch.argmax(model(inputs["input_ids"], inputs["attention_mask"]), dim=1)

def custom_embedding_function(inputs):
    return model.l1(inputs["input_ids"], inputs["attention_mask"])[0][:, 0, :]  # [CLS] token embedding

def custom_soft_classification_function(inputs):
    return model(inputs["input_ids"], inputs["attention_mask"])


In [16]:
wrapper = Wrapper(
    train_df=train_df, 
    test_df=test_df[:2000], 
    encodings=encodings, 
    input_col_name="text", 
    target_col_name="target", 
    cache_path="data/custom_cache",
    overwrite=False
)
wrapper.encode_function = custom_encoding_function
wrapper.classification_function = custom_classification_function
wrapper.soft_classification_function = custom_soft_classification_function
wrapper.embedding_function = custom_embedding_function
wrapper.prepare()

INFO:root:Checking for cached predictions...
INFO:root:cached version 'data/custom_cache/predictions.json' found
INFO:root:Checking for cached embeddings...
INFO:root:cached version 'data/custom_cache/embedding_training.json' found
INFO:root:cached version 'data/custom_cache/embedding_testing.json' found
INFO:root:Checking for cached projections...
INFO:root:cached version 'data/custom_cache/projections_training.json' found
INFO:root:cached version 'data/custom_cache/projections_testing.json' found
INFO:root:cached version 'data/custom_cache/projector.pkl.gz' found
INFO:root:Checking for cached salience maps...
INFO:root:cached version 'data/custom_cache/salience.pkl.gz' found
INFO:root:Checking for cached cluster profiles...
INFO:root:cached version 'data/custom_cache/cluster_profiles.pkl.gz' found
INFO:root:cached version 'data/custom_cache/clusters.json' found
INFO:root:cached version 'data/custom_cache/cluster_labels.json' found
INFO:root:cached version 'data/custom_cache/cluster_w

In [23]:
%matplotlib agg
dash = Dashboard(wrapper, show_wordclouds=True)
dash.render()

To play with different UMAP and DBSCAN arguments without having to recompute the entire `prepare()` function, you can use `recompute_projections` (which will recompute both the projections and visual clusterings) or `recompute_visual_clusterings` (which will only redo the clustering)

In [18]:
# wrapper.recompute_visual_clusterings(dbscan_args=dict())
# wrapper.recompute_projections(umap_args=dict(min_dist=.2), dbscan_args=dict())

To test or debug the classification model/see what raw outputs the viusualizations are getting, or create your own visualization tools, you can manually call the `classify()`, `soft_classify()`, `embed()` functions, or get access to any of the cached data as seen in the bottom cell

In [19]:
wrapper.classify(["testing"])

[15]

In [20]:
wrapper.soft_classify(["testing"])

[[-1.1586105823516846,
  -0.269292950630188,
  0.32233911752700806,
  0.05956360697746277,
  0.470281183719635,
  -0.1995173841714859,
  0.812308669090271,
  0.11527691781520844,
  -0.5965903997421265,
  -0.0914115458726883,
  -0.3710917830467224,
  -0.022948797792196274,
  0.31187009811401367,
  1.1063690185546875,
  -0.029917806386947632,
  0.11398419737815857,
  -0.5523065328598022,
  -0.5899699330329895,
  0.39079421758651733,
  -0.7065202593803406]]

In [21]:
wrapper.embed(["testing"])

[[0.17447194457054138,
  0.9931786060333252,
  -0.01606195978820324,
  -1.1096956729888916,
  -0.3842974305152893,
  0.5603211522102356,
  0.14243218302726746,
  0.15506093204021454,
  -0.6247150301933289,
  -1.0153950452804565,
  -0.06342215836048126,
  1.103736400604248,
  0.04490356147289276,
  1.1352771520614624,
  -0.8774974942207336,
  -0.5332215428352356,
  0.1610693484544754,
  0.3871626853942871,
  1.369088053703308,
  0.40271803736686707,
  0.030687661841511726,
  -0.8779581189155579,
  0.9278408288955688,
  0.008592884987592697,
  -0.29590949416160583,
  -0.8164458274841309,
  0.37475383281707764,
  0.6363185048103333,
  0.9378421902656555,
  0.6158270239830017,
  -0.2505626678466797,
  0.20758314430713654,
  -0.5005321502685547,
  0.2038734406232834,
  -0.3629128336906433,
  0.08684615045785904,
  0.41998451948165894,
  0.3224567770957947,
  -0.5328847765922546,
  -0.13581889867782593,
  -1.3324553966522217,
  -0.08679085969924927,
  -0.041342999786138535,
  0.1468113064765

In [22]:
# cached data:
# wrapper.embeddings_training
# wrapper.embeddings_testing
# wrapper.projector
# wrapper.projections_training
# wrapper.projections_testing
# wrapper.salience_maps
# wrapper.clusters
# wrapper.cluster_profiles
# wrapper.cluster_words_freqs
# wrapper.cluster_class_word_sets